
# **Dùng mô hình T5-small đã huấn luyện**
---


## 1.Chuẩn bị môi trường

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
# install pytorch with GPU accelerated
# (see https://pytorch.org/get-started/locally/ )
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu114

# install sentencepiece for lingual modeling
!pip install omegaconf hydra-core fairseq sentencepiece

# install huggingface libraries
!pip install transformers datasets evaluate

# install additional packages
!pip install protobuf==3.20.3
!pip install absl-py rouge_score nltk
!pip install numpy

# install jupyter if you run code in notebook
!pip install jupyter
!pip install datasets transformers==4.28.0
!pip install --upgrade accelerate
!pip install bert_score
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu114
  Using cached omegaconf-2.3.0-py3-none-any.whl (79 kB)
  Using cached hydra_core-1.3.2-py3-none-any.whl (154 kB)
  Using cached fairseq-0.12.2.tar.gz (9.6 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached antlr4-python3-runtime-4.9.3.tar.gz (117 kB)
  Preparing metadata (setup.py) ... done
  Using cached hydra_core-1.0.7-py3-none-any.whl (123 kB)
INFO: pip is looking at multiple versions of fairseq to determine which version is compatible with other requirements. This could take a while.
  Using cached fairseq-0.12.1.tar.gz (9.6 MB)
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates fro

  Using cached protobuf-3.20.2-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.1 MB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.
tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.


In [11]:
import os
# drive path to save check point and load trained file
content="/content/drive/MyDrive/HTK"

## **2.Tải Dataset**
*Model T5 ban đầu sử dụng dataset Colossal Clean Crawled Corpus (C4)* chi tiết tại :https://jmlr.org/papers/volume21/20-074/20-074.pdf

Link dataset Colossal Clean Crawled Corpus (C4): https://www.tensorflow.org/datasets/catalog/c4?hl=vi

Trong bài này nhóm em thực hiện chọn dataset xlsum để tái huấn luyện dữ liệu.

Vì lý do phần cứng nên nhóm em thực hiện giảm 50% lượng dữ liệu để có thể huấn luyện.

In [12]:
from datasets import load_dataset,DatasetDict

#load dataset with 50% of each type "train","test","validation"
ds_train = load_dataset("csebuetnlp/xlsum", name="english",split="train[:5%]")
ds_test = load_dataset("csebuetnlp/xlsum", name="english",split="test[:5%]")
ds_val = load_dataset("csebuetnlp/xlsum", name="english",split="validation[:5%]")

#create dataset dictionary
ds = DatasetDict({"train":ds_train,"test":ds_test,"validation":ds_val})
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 15326
    })
    test: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 577
    })
    validation: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 577
    })
})

In [13]:
ds["train"][0]

{'id': 'uk-wales-56321577',
 'url': 'https://www.bbc.com/news/uk-wales-56321577',
 'title': 'Weather alert issued for gale force winds in Wales',
 'summary': 'Winds could reach gale force in Wales with stormy weather set to hit the whole of the country this week.',
 'text': 'The Met Office has issued a yellow weather warning for wind covering Wales and England, starting from 21:00 GMT on Wednesday evening. Travel and power are both likely to be disrupted, with the warning to remain in place until 15:00 on Thursday. Gusts of 55mph (88kmh) are likely and could hit up to 70mph on coasts and hills, with heavy and blustery showers.'}

## **3.Chuẩn bị model**

### **3.1 Chuẩn bị T5 tokenizer**

In [14]:
from transformers import AutoTokenizer
t5_tokenizer = AutoTokenizer.from_pretrained("t5-small",use_fast=False)

In [15]:
def tokenize_sample_data(data):
  # Max token size is 14536 and 215 for inputs and labels, respectively.
  # Here I restrict these token size.
  # We limited max_length of token to saving memory and computing resources and time, which can improve performance and avoid out of memory
  input_feature = t5_tokenizer(data["text"], truncation=True, max_length=1024)
  label = t5_tokenizer(data["summary"], truncation=True, max_length=128)
  return {
    "input_ids": input_feature["input_ids"],
    "attention_mask": input_feature["attention_mask"],
    "labels": label["input_ids"],
  }

tokenized_ds = ds.map(
  tokenize_sample_data,
  remove_columns=["id", "url", "title", "summary", "text"],
  batched=True,
  batch_size=128)

tokenized_ds

Map:   0%|          | 0/15326 [00:00<?, ? examples/s]

Map:   0%|          | 0/577 [00:00<?, ? examples/s]

Map:   0%|          | 0/577 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 15326
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 577
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 577
    })
})

### **3.2 Chuẩn bị model**

Load model từ file trained

In [16]:
import torch
from transformers import AutoConfig, AutoModelForSeq2SeqLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#see  https://huggingface.co/docs/transformers/v4.30.0/en/main_classes/configuration#transformers.PretrainedConfig
#load pretrain model and config
mt5_config = AutoConfig.from_pretrained(
  "t5-small",
  max_length=128,
  length_penalty=0.6, # Exponential penalty to the length that is used with beam-based generation,length_penalty > 0.0 promotes longer sequences, while length_penalty < 0.0 encourages shorter sequences.
  no_repeat_ngram_size=2,
  num_beams=15,
)
model = (AutoModelForSeq2SeqLM
         .from_pretrained(  "t5-small", config=mt5_config)
         .to(device))

In [17]:
from transformers import DataCollatorForSeq2Seq
# see https://huggingface.co/docs/transformers/v4.30.0/en/main_classes/data_collator#transformers.DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(
  t5_tokenizer,
  model=model,
  return_tensors="pt")

In [18]:
import os
from transformers import AutoModelForSeq2SeqLM
# load local model
model = (AutoModelForSeq2SeqLM
         .from_pretrained(os.path.join(content,"trained_t5_small_summarize"))
         .to(device))

## **4.Dự đoán bằng dataset**

In [19]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [21]:
from torch.utils.data import DataLoader
import numpy as np
# Predict with test data (first 5 rows)
sample_dataloader = DataLoader(
  tokenized_ds["test"].with_format("torch"),
  collate_fn=data_collator,
  batch_size=5)
for batch in sample_dataloader:
  with torch.no_grad():
    preds = model.generate(
      batch["input_ids"].to(device),
      num_beams=15,
      num_return_sequences=1,
      no_repeat_ngram_size=1,
      remove_invalid_values=True,
      max_length=128,
    )
  labels = batch["labels"]
  break

# Replace -100 (see above)
labels = np.where(labels != -100, labels, t5_tokenizer.pad_token_id)

# Convert id tokens to text
text_preds = t5_tokenizer.batch_decode(preds, skip_special_tokens=True)
text_labels = t5_tokenizer.batch_decode(labels, skip_special_tokens=True)



In [22]:
# Show result
print("***** Input's Text *****")
print(ds["test"]["text"][1])
print("***** Summary Text (True Value) *****")
print(text_labels[1])
print("***** Summary Text (Generated Text) *****")
print(text_preds[1])

***** Input's Text *****
But Eluned Morgan conceded that it would be "difficult for us to stop" from a legal point of view. Her comments were criticised by a Labour AM. Alun Davies said threatening legal action "sounds like the last breath before you're thrown out of the pub". Mr Davies said he was not convinced the Welsh Government would "have a leg to stand on" in trying to shape international trade deals after Brexit. Following Donald Trump's comments during last week's trade visit that the NHS would be "on the table" in any future trade talks between the UK and the USA, Eluned Morgan said there was "absolutely no prospect whatsoever of us allowing the Welsh NHS to be part of any negotiation." The US President then rowed back on his initial comments following criticism from a number of MPs. Asked about her response to President Trump's remarks as she gave evidence to the Assembly's Brexit committee on Monday, Ms Morgan said "legally, it would be difficult for us to stop because we d

## **6.Dự đoán bằng input**

In [23]:
inpt_text="Prime Minister Pham Minh Chinh wants to look into the possibility of developing a standard, high-speed railway that connects Vietnam and China. At a meeting with Chinese President Xi Jinping at the Great Hall of the People in China's Beijing on Tuesday, Chinh requested China to bolster the progress of opening the market for Vietnam's agricultural products, and create opportunities for Vietnam to open more offices to promote trade in China. Chinh also wants China to give more quota for Vietnamese goods in transit to a third country by China’s railway, as well as looking into cooperation possibilities for a standard, high-speed railway connecting the two countries. Chinh also commended Chinese businesses for expanding high-quality investments into Vietnam, wishing for more exchanges between both countries and to contribute to a resilient social foundation for bilateral relations. Chinh stressed that a stable and long-term relations with China has always been a strategic choice of foremost priority for Vietnam's external relations. In response, Xi said China has always regarded Vietnam as a priority when it comes to its diplomatic policies, and wants to bolster the relationship between the two countries and the two Parties.Xi said China is ready to maintain strategic exchanges with Vietnam, import goods from Vietnam and bolster railway, road and border infrastructure connectivity. Xi commends Vietnam for participating in China's global initiatives and promoting peace, cooperation and development in the region and the world."

In [24]:

input_feature = t5_tokenizer(inpt_text, truncation=True, max_length=1024, return_tensors="pt")
preds = model.generate(
      input_feature["input_ids"].to(device),
      num_beams=15,
      num_return_sequences=1,
      no_repeat_ngram_size=1,
      remove_invalid_values=True,
      max_length=128,
    )

In [25]:
text_pr = t5_tokenizer.batch_decode(preds, skip_special_tokens=True)
print("***** Input's Text *****")
print(inpt_text)
print("***** Summary Text (Generated Text) *****")
print(text_pr)

***** Input's Text *****
Prime Minister Pham Minh Chinh wants to look into the possibility of developing a standard, high-speed railway that connects Vietnam and China. At a meeting with Chinese President Xi Jinping at the Great Hall of the People in China's Beijing on Tuesday, Chinh requested China to bolster the progress of opening the market for Vietnam's agricultural products, and create opportunities for Vietnam to open more offices to promote trade in China. Chinh also wants China to give more quota for Vietnamese goods in transit to a third country by China’s railway, as well as looking into cooperation possibilities for a standard, high-speed railway connecting the two countries. Chinh also commended Chinese businesses for expanding high-quality investments into Vietnam, wishing for more exchanges between both countries and to contribute to a resilient social foundation for bilateral relations. Chinh stressed that a stable and long-term relations with China has always been a st